## Havoc on the Hill -- A Visualisation of Canadian Legislation Since 1994
By Riley Wheadon  
January 2nd, 2023
---

In [19]:
# Import Python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
# Read .json file
data = pd.read_json('../data/raw/bills.json')

# Display first three rows of the dataframe
display(data.head(3))

,Id,NumberCode,NumberPrefix,Number,LongTitleEn,LongTitleFr,LongTitle,ShortTitleEn,ShortTitleFr,ShortTitle,...,HouseVoteDetails,SenateVoteDetails,HouseRulingAndStatements,SenateRulingAndStatements,WebReferences,BibliographicNotices,SenatePreStudyCommitteeDetails,SimilarBills,IsDroppedFromSenateOrderPaper,IsSessionOngoing
0,11473483,S-1,NaN,0,An Act relating to railways,Loi concernant les chemins de fer,An Act relating to railways,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
1,10867804,S-1,NaN,0,An Act relating to railways,Loi concernant les chemins de fer,An Act relating to railways,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
2,10601138,S-1,NaN,0,An Act relating to railways,Loi concernant les chemins de fer,An Act relating to railways,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [21]:
# Create helper function to print missing values by column
def printNull(df):
    output = "\n".join([
        "-----------", 
        "{shape}",
        "-----------",
        "{missing}"
    ]).format(
        shape = df.shape, 
        missing = df.isna().sum().to_string()
    )
    print(output)

# Print data shape, column names, and missing values
printNull(data)

-----------
(6761, 135)
-----------
Id                                                   0
NumberCode                                           0
NumberPrefix                                      6761
Number                                               0
LongTitleEn                                          0
LongTitleFr                                          0
LongTitle                                            0
ShortTitleEn                                         0
ShortTitleFr                                         0
ShortTitle                                           0
StatusId                                          6761
StatusNameEn                                         0
StatusNameFr                                         0
StatusName                                           0
LatestCompletedMajorStageId                          0
LatestCompletedMajorStageNameEn                      0
LatestCompletedMajorStageNameFr                      0
LatestCompletedMajorStageName

As shown above, the dataset has a staggering **135** columns. However, multiple columns are missing all 6761 rows of data. We'll start by removing these columns. Additionally, some columns are redundant (i.e. duplicate information in French).

---

In [22]:
# Remove columns containing missing all values
data.dropna(axis = 1, how = 'all', inplace = True)

# Remove columns containing all zeros
data = data.loc[:, (data != 0).any(axis = 0)]

# Remove all columns that end in "En" or "Fr" to eliminate redundancies
data = data.loc[:, [col for col in data.columns if col[-2:] not in ("En", "Fr")]]

# Print information about new data after column removal
printNull(data)

-----------
(6761, 34)
-----------
Id                                                   0
NumberCode                                           0
LongTitle                                            0
ShortTitle                                           0
StatusName                                           0
LatestCompletedMajorStageId                          0
LatestCompletedMajorStageName                        0
LatestCompletedMajorStageNameWithChamberSuffix       0
LatestCompletedMajorStageChamberOrganizationId       0
ParliamentNumber                                     0
SessionNumber                                        0
BillDocumentTypeName                                 0
OriginatingChamberOrganizationId                     0
IsHouseBill                                          0
IsSenateBill                                         0
SponsorPersonName                                    0
LatestBillEventDateTime                              0
PassedHouseFirstReadingDateTim

It appears that the data contains groups of columns with binary data (i.e. PassedFirstChamberFirstReading and similar columns). These groups can be replaced with a single categorical column to improve clarity.

In [23]:
# Aggregate binary 'reading passed' columns into a single column
reading_columns = [col for col in data.columns if col[-7:] == "Reading"]
data["ReadingsPassed"]  = data[reading_columns].astype(int).sum(1)

# Aggregate IsHouseBill and IsSenateBill
data["BillOrigin"] = data[['IsHouseBill', 'IsSenateBill']].idxmax(1)

# Unify the title columns, with a preference for short titles where possible
data['ShortTitle'] = np.where(data['ShortTitle'] == '', data['LongTitle'], data['ShortTitle'])

# Drop initial aggregated columns
data.drop(columns = reading_columns + ['IsHouseBill', 'IsSenateBill', 'LongTitle'], inplace = True)

# Display all remaining columns
with pd.option_context('display.max_columns', None): 
    display(data.head(3))

,Id,NumberCode,ShortTitle,StatusName,LatestCompletedMajorStageId,LatestCompletedMajorStageName,LatestCompletedMajorStageNameWithChamberSuffix,LatestCompletedMajorStageChamberOrganizationId,ParliamentNumber,SessionNumber,BillDocumentTypeName,OriginatingChamberOrganizationId,SponsorPersonName,LatestBillEventDateTime,PassedHouseFirstReadingDateTime,PassedHouseSecondReadingDateTime,PassedHouseThirdReadingDateTime,PassedSenateFirstReadingDateTime,PassedSenateSecondReadingDateTime,PassedSenateThirdReadingDateTime,ReceivedRoyalAssentDateTime,DidReinstateFromPreviousSession,ReceivedRoyalAssent,BillStages,IsSessionOngoing,ReadingsPassed,BillOrigin
0,11473483,S-1,An Act relating to railways,Introduced as pro forma bill,60043,First reading,First reading in the Senate,2,44,1,Senate Public Bill,2,,0001-01-01T00:00:00,None,None,None,2021-11-22T19:00:00-05:00,None,None,None,False,False,"{'BillId': 0, 'HouseBillStages': [], 'SenateBi...",True,1,IsSenateBill
1,10867804,S-1,An Act relating to railways,Introduced as pro forma bill,60043,First reading,First reading in the Senate,2,43,2,Senate Public Bill,2,,0001-01-01T00:00:00,None,None,None,2020-09-22T20:00:00-04:00,None,None,None,False,False,"{'BillId': 0, 'HouseBillStages': [], 'SenateBi...",False,1,IsSenateBill
2,10601138,S-1,An Act relating to railways,Introduced as pro forma bill,60043,First reading,First reading in the Senate,2,43,1,Senate Public Bill,2,,0001-01-01T00:00:00,None,None,None,2019-12-04T19:00:00-05:00,None,None,None,False,False,"{'BillId': 0, 'HouseBillStages': [], 'SenateBi...",False,1,IsSenateBill


In [24]:
# There are still some more redundant columns, which will be removed manually
data.drop(
    columns = [
        'LatestCompletedMajorStageId', 'LatestCompletedMajorStageName', 'LatestCompletedMajorStageChamberOrganizationId',
        'LatestBillEventDateTime', 'OriginatingChamberOrganizationId', 'DidReinstateFromPreviousSession', 'BillStages', 
        'StatusName', 'SponsorPersonName'
    ], 
    inplace = True
)

# Rename some columns for clarity
data.rename(
    columns = {
        'NumberCode': 'Code',
        'ShortTitle': 'Title',
        'LatestCompletedMajorStageNameWithChamberSuffix': 'LatestStageName',
        'BillDocumentTypeName': 'BillType',
        'IsSessionOngoing': 'Ongoing'
    }, 
    inplace = True
)

# Display remaining columns
display(data.head(3))

,Id,Code,Title,LatestStageName,ParliamentNumber,SessionNumber,BillType,PassedHouseFirstReadingDateTime,PassedHouseSecondReadingDateTime,PassedHouseThirdReadingDateTime,PassedSenateFirstReadingDateTime,PassedSenateSecondReadingDateTime,PassedSenateThirdReadingDateTime,ReceivedRoyalAssentDateTime,ReceivedRoyalAssent,Ongoing,ReadingsPassed,BillOrigin
0,11473483,S-1,An Act relating to railways,First reading in the Senate,44,1,Senate Public Bill,None,None,None,2021-11-22T19:00:00-05:00,None,None,None,False,True,1,IsSenateBill
1,10867804,S-1,An Act relating to railways,First reading in the Senate,43,2,Senate Public Bill,None,None,None,2020-09-22T20:00:00-04:00,None,None,None,False,False,1,IsSenateBill
2,10601138,S-1,An Act relating to railways,First reading in the Senate,43,1,Senate Public Bill,None,None,None,2019-12-04T19:00:00-05:00,None,None,None,False,False,1,IsSenateBill


In [25]:
# Get DateTime columns
datetime_columns = [col for col in data.columns if col[-8:] == "DateTime"]
data[datetime_columns] = data[datetime_columns].apply(lambda x: pd.to_datetime(x.str[:10]))

# Create new columns and calculate the amount of time the bill was debated
data["FirstStageDate"] = data[datetime_columns].min(axis = 1)
data["LastStageDate"] = data[datetime_columns].max(axis = 1)
data["TimeAlive"] = data["LastStageDate"] - data["FirstStageDate"]

# Drop original DateTime columns
data.drop(columns = datetime_columns, inplace = True)

# Replace Id column with a commonly used combination of parliament number, sesion number, and code
data["Id"] = data["ParliamentNumber"].astype(str) + "-" + data["SessionNumber"].astype(str) + "/" +  data["Code"] 

# Print remaining columns and check for null values
printNull(data)
display(data.head(3))

-----------
(6761, 14)
-----------
Id                       0
Code                     0
Title                    0
LatestStageName          0
ParliamentNumber         0
SessionNumber            0
BillType                 0
ReceivedRoyalAssent      0
Ongoing                  0
ReadingsPassed           0
BillOrigin               0
FirstStageDate         975
LastStageDate          975
TimeAlive              975


,Id,Code,Title,LatestStageName,ParliamentNumber,SessionNumber,BillType,ReceivedRoyalAssent,Ongoing,ReadingsPassed,BillOrigin,FirstStageDate,LastStageDate,TimeAlive
0,44-1/S-1,S-1,An Act relating to railways,First reading in the Senate,44,1,Senate Public Bill,False,True,1,IsSenateBill,2021-11-22,2021-11-22,0 days
1,43-2/S-1,S-1,An Act relating to railways,First reading in the Senate,43,2,Senate Public Bill,False,False,1,IsSenateBill,2020-09-22,2020-09-22,0 days
2,43-1/S-1,S-1,An Act relating to railways,First reading in the Senate,43,1,Senate Public Bill,False,False,1,IsSenateBill,2019-12-04,2019-12-04,0 days


The data has been reduced down to a reasonable size, but there are still some issues. The PersonName column is filled with whitespace and there is no information on political affiliations of the various bills. Webscraping with beautifulsoup4 will be used to get additional data.

--- 

In [26]:
# Export cleaned data to a .csv file
data.to_csv('../data/processed/bills_processed.csv', index = False)